In [1]:
#numpy and pandas for data manipulation
import numpy as np
import pandas as pd
import gc

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_predict

In [2]:
samples = pd.read_csv('training.csv')

In [3]:
samples.shape

(307507, 1331)

In [4]:
# def convert_types(df, print_info = False):
    
#     original_memory = df.memory_usage().sum()
    
#     # Iterate through each column
#     for c in df:
        
#         # Convert ids and booleans to integers
#         if ('SK_ID' in c):
#             df[c] = df[c].fillna(0).astype(np.int32)
            
#         # Convert objects to category
#         elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
#             df[c] = df[c].astype('category')
        
#         # Booleans mapped to integers
#         elif list(df[c].unique()) == [1, 0]:
#             df[c] = df[c].astype(bool)
        
#         # Float64 to float32
#         elif df[c].dtype == float:
#             df[c] = df[c].astype(np.float32)
            
#         # Int64 to int32
#         elif df[c].dtype == int:
#             df[c] = df[c].astype(np.int32)
        
#     new_memory = df.memory_usage().sum()
    
#     if print_info:
#         print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
#         print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
#     return df
        
        
# samples = convert_types(samples)

In [5]:
round(samples.memory_usage().sum() / 1e9, 2)

3.25

In [6]:
EMPTY_THRESHOLD = 75
CORR_THRESHOLD = 0.9

In [7]:
def remove_missing_columns(train, threshold = 90):
    # Calculate missing stats for train
    train_miss = pd.DataFrame(train.isnull().sum())
    train_miss['percent'] = 100 * train_miss[0] / len(train)
    
    # list of missing columns for train and test
    missing_train_columns = list(train_miss.index[train_miss['percent'] > threshold])
    
    # Combine the two lists together
    missing_columns = list(set(missing_train_columns))
    
    # Print information
    print('There are %d columns with greater than %d%% missing values.' % (len(missing_columns), threshold))
    
    # Drop the missing columns and return
    return train.drop(columns = missing_columns)

samples = remove_missing_columns(samples, EMPTY_THRESHOLD)


There are 69 columns with greater than 75% missing values.


In [8]:
samples.shape

(307507, 1262)

In [9]:
samples['TARGET']

0          True
1         False
2         False
3         False
4         False
          ...  
307502    False
307503    False
307504    False
307505     True
307506    False
Name: TARGET, Length: 307507, dtype: bool

In [10]:
correlations = samples.drop(columns = ['TARGET']).corr().abs()


In [11]:
for c in samples.columns:
    if 'TARGET' in c:
        print(c)

TARGET


In [14]:
# Select upper triangle of correlation matrix
upper = correlations.where(np.triu(np.ones(correlations.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > CORR_THRESHOLD)]

In [15]:
len(to_drop)

532

In [19]:
samples = samples.drop(columns=to_drop)

In [29]:
samples.shape

(307507, 730)

In [21]:
samples.to_csv('preprocessed-75.csv',index=False)

In [27]:
remove_missing_columns(samples, 65).to_csv('preprocessed-65.csv', index=False)

There are 152 columns with greater than 65% missing values.


In [28]:
remove_missing_columns(samples, 50).to_csv('preprocessed-50.csv', index=False)

There are 161 columns with greater than 50% missing values.


In [25]:
remove_missing_columns(samples, 50)

There are 161 columns with greater than 50% missing values.


,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,...,client_installments_AMT_INSTALMENT_max_mean,client_installments_AMT_PAYMENT_sum_min,client_installments_AMT_INSTALMENT_max_max,client_installments_AMT_PAYMENT_sum_mean,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_sum_sum
0,True,0,True,0,True,0,202500.0,406597.5,24700.5,Unaccompanied,...,53093.746,219625.700,53093.746,219625.700,175783.73,175783.73,219625.700,219625.690,1.008781e+06,4.172888e+06
1,False,0,False,0,False,0,270000.0,1293502.5,35698.5,Family,...,175764.050,80773.380,560835.400,453952.220,1154108.20,1154108.20,1150977.400,1618864.600,4.394102e+06,1.134881e+07
2,False,1,True,1,True,0,67500.0,135000.0,6750.0,Unaccompanied,...,10573.965,21288.465,10573.965,21288.465,16071.75,16071.75,21288.465,21288.465,3.172190e+04,6.386539e+04
3,False,0,False,0,True,0,135000.0,312682.5,29686.5,Unaccompanied,...,66116.266,25091.324,691786.900,232499.700,994476.70,994476.70,691786.900,1007153.400,1.057860e+06,3.719995e+06
4,False,0,True,0,True,0,121500.0,513000.0,21865.5,Unaccompanied,...,12677.324,18330.390,22678.785,172669.890,483756.38,825845.80,280199.700,806127.940,8.367034e+05,1.139621e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307502,False,0,True,0,False,0,157500.0,254700.0,27558.0,Unaccompanied,...,12815.010,52450.470,12815.010,52450.470,46241.37,46241.37,52450.470,52450.470,8.970507e+04,3.671533e+05
307503,False,0,False,0,True,0,72000.0,269550.0,12001.5,Unaccompanied,...,10074.465,60419.203,10074.465,60419.203,60281.28,60281.28,60419.203,60419.203,6.044679e+04,3.625152e+05
307504,False,0,False,0,True,0,153000.0,677664.0,29979.0,Unaccompanied,...,4404.410,8317.890,5575.185,21778.004,41805.99,61529.04,33413.758,57622.812,6.166175e+04,3.048920e+05
307505,True,0,False,0,True,0,171000.0,370107.0,20205.0,Unaccompanied,...,10239.832,22964.398,19065.824,93367.150,194556.81,194556.81,171592.420,194556.810,1.945568e+05,1.773976e+06


In [26]:
to_drop

['AMT_GOODS_PRICE',
 'REGION_RATING_CLIENT_W_CITY',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAREA_MEDI',
 'TOTALAREA_MODE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DAYS_EMPLOYED_ANOM',
 'DAYS_EMPLOYED_PERCENT',
 'bureau_DAYS_ENDDATE_FACT_sum',
 'bureau_DAYS_CREDIT_UPDATE_sum',
 'bureau_CREDIT_DAY_OVERDUE_sum',
 'bureau_AMT_CREDIT_SUM_OVERDUE_sum',
 'bureau_CNT_CREDIT_PROLONG_sum',
 'bureau_AMT_CREDI